# Multi-table Datasets - ENRON Archive

## 1. Data import

Connect to the file 'assets/datasets/enron.db' using one of these methods:

- sqlite3 python package
- pandas.read_sql
- SQLite Manager Firefox extension

Take a look at the database and query the master table. How many Tables are there in the db?

> Answer:
There are 3 tables:
- MessageBase
- RecipientBase
- EmployeeBase

In [69]:
import pandas as pd
from pandas.io import sql
import sqlite3
sqlite_db = '/Users/Rebecca/DSI-projects/curriculum/week05/1.3-lab-sqlite-enron/assets/datasets/enron.db'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()
import matplotlib.pyplot as plt
import numpy as np
from bokeh import mpl
from bokeh.plotting import output_file, show

Query the `sqlite_master` table to retrieve the schema of the `EmployeeBase` table.

1. What fields are there?
1. What's the type of each of them?

In [25]:
sql.read_sql('SELECT * from sqlite_master', con = conn)

,type,name,tbl_name,rootpage,sql
0,view,Employee,Employee,0,"CREATE VIEW Employee AS\nSELECT\n eid,\n ..."
1,view,EmployeeWithVars,EmployeeWithVars,0,CREATE VIEW EmployeeWithVars AS\nSELECT\n e...
2,table,MessageBase,MessageBase,3,"CREATE TABLE MessageBase (\n mid INTEGER,\n..."
3,index,IX_MessageBase_unix_time,MessageBase,4,CREATE INDEX IX_MessageBase_unix_time ON Messa...
4,view,Message,Message,0,"CREATE VIEW Message AS\nSELECT\n mid,\n ..."
5,table,RecipientBase,RecipientBase,5,"CREATE TABLE RecipientBase (\n mid INTEGER,..."
6,index,sqlite_autoindex_RecipientBase_1,RecipientBase,6,None
7,view,Recipient,Recipient,0,"CREATE VIEW Recipient AS\nSELECT\n mid,\n ..."
8,table,EmployeeBase,EmployeeBase,2,CREATE TABLE EmployeeBase (\n ...


In [23]:
sql.read_sql('SELECT * FROM EmployeeBase limit 0', con = conn)

,eid,name,department,longdepartment,title,gender,seniority


In [26]:
sql.read_sql('SELECT * FROM EmployeeBase limit 0', con = conn).dtypes

eid               object
name              object
department        object
longdepartment    object
title             object
gender            object
seniority         object
dtype: object

1. Print the first 5 rows of EmployeeBase table
1. Print the first 5 rows of MessageBase table
1. Print the first 5 rows of RecipientBase table

**Hint**  use `SELECT` and `LIMIT`.

In [27]:
sql.read_sql('SELECT * FROM EmployeeBase limit 5', con = conn)

,eid,name,department,longdepartment,title,gender,seniority
0,1,John Arnold,Forestry,ENA Gas Financial,VP Trading,Male,Senior
1,2,Harry Arora,Forestry,ENA East Power,VP Trading,Male,Senior
2,3,Robert Badeer,Forestry,ENA West Power,Mgr Trading,Male,Junior
3,4,Susan Bailey,Legal,ENA Legal,Specialist Legal,Female,Junior
4,5,Eric Bass,Forestry,ENA Gas Texas,Trader,Male,Junior


In [28]:
sql.read_sql('SELECT * FROM MessageBase limit 5', con = conn)

,mid,filename,unix_time,subject,from_eid
0,1,taylor-m/sent/11,910930020,Cd$ CME letter,138
1,2,taylor-m/sent/17,911459940,Indemnification,138
2,3,taylor-m/sent/18,911463840,Re: Indemnification,138
3,4,taylor-m/sent/23,911874180,"Re: Coral Energy, L.P.",138
4,5,taylor-m/sent/27,912396120,Bankruptcy Code revisions,138


In [29]:
sql.read_sql('SELECT * FROM RecipientBase limit 5', con = conn)

,mid,rno,to_eid
0,1,1,59
1,2,1,15
2,3,1,15
3,4,1,109
4,4,2,49


Import each of the 3 tables to a Pandas Dataframes

In [33]:
df_e = sql.read_sql('SELECT * FROM EmployeeBase', con = conn)
df_m = sql.read_sql('SELECT * FROM MessageBase', con = conn)
df_r = sql.read_sql('SELECT * FROM RecipientBase', con = conn)

## 2. Data Exploration

Use the 3 dataframes to answer the following questions:

1. How many employees are there in the company?
- How many messages are there in the database?
- Convert the timestamp column in the messages. When was the oldest message sent? And the newest?
- Some messages are sent to more than one recipient. Group the messages by message_id and count the number of recepients. Then look at the distribution of recepient numbers.
    - How many messages have only one recepient?
    - How many messages have >= 5 recepients?
    - What's the highest number of recepients?
    - Who sent the message with the highest number of recepients?
- Plot the distribution of recepient numbers using Bokeh.

In [68]:
print '1. There are %s employees in the company.' %len(df_e)
print '2. There are %s messages in the database.' %len(df_m)
df_m['time'] = pd.to_datetime(df_m['unix_time'], unit = 's')
df_m.sort_values('time', ascending = True, inplace = True)
print '3. The earliest message was sent on %s' %min(df_m['time']) +'; the latest message was sent on %s.' %max(df_m['time'])
msgs = df_r.groupby(['mid'])['rno'].count().to_frame('rno_count').sort_values('rno_count', ascending = False)
print '4. There are %s' %len(msgs[msgs['rno_count'] == 1]) + ' messages with one recipient.'
print '5. There are %s' %len(msgs[msgs['rno_count'] >= 5]) + ' messages with at least five recipients.'
print '6. The maximum number of recipients is %s.' %max(msgs['rno_count'])
print '7. Sender number %s' %int(df_m[df_m['mid'] == 12116]['from_eid'].values) + ' sent the message with the most recipients.'

1. There are 156 employees in the company.
2. There are 21635 messages in the database.
3. The earliest message was sent on 1998-11-13 04:07:00; the latest message was sent on 2002-06-21 13:37:34.
4. There are 14985 messages with one recipient.
5. There are 1380 messages with at least five recipients.
6. The maximum number of recipients is 57.
7. Sender number 67 sent the message with the most recipients.


In [72]:
new_df = df_r.groupby(['mid'])['rno'].count()

pandas.core.series.Series

In [74]:
plt.plot(new_df)
plt.title("Matplotlib Figure in Bokeh")

output_file("mpl_plot.html", title="SO COOWL")

show(mpl.to_bokeh())

Rescale to investigate the tail of the curve

## 3. Data Merging

Use the pandas merge function to combine the information in the 3 dataframes to answer the following questions:

1. Are there more Men or Women employees?
- How is gender distributed across departments?
- Who is sending more emails? Men or Women?
- What's the average number of emails sent by each gender?
- Are there more Juniors or Seniors?
- Who is sending more emails? Juniors or Seniors?
- Which department is sending more emails? How does that relate with the number of employees in the department?
- Who are the top 3 senders of emails? (people who sent out the most emails)

Answer the following questions regarding received messages:

- Who is receiving more emails? Men or Women?
- Who is receiving more emails? Juniors or Seniors?
- Which department is receiving more emails? How does that relate with the number of employees in the department?
- Who are the top 5 receivers of emails? (people who received the most emails)

Which employees sent the most 'mass' emails?

Keep exploring the dataset, which other questions would you ask?

Work in pairs. Give each other a challenge and try to solve it.